In [9]:
from pydub import AudioSegment
import re
AudioSegment.converter = r"C:\Users\abelf\ffmpeg-N-104359-g9b445663a5-win64-lgpl\ffmpeg-N-104359-g9b445663a5-win64-lgpl\bin\ffmpeg.exe"
AudioSegment.ffprobe =  r"C:\Users\abelf\ffmpeg-N-104359-g9b445663a5-win64-lgpl\ffmpeg-N-104359-g9b445663a5-win64-lgpl\bin\ffprobe.exe"
import math
from pydub import AudioSegment
import math
import matplotlib.pyplot as plt
import os
import numpy as np
import librosa
import pandas as pd
import openpyxl
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '\\' + filename
        
        self.audio = AudioSegment.from_mp3(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(r"C:\Users\abelf\Downloads\songs-upbeat - divided"+ '\\' + split_filename, format="mp3")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 20)
        for i in range(0, total_mins, min_per_split):
            split_fn =  re.sub(r'[0-9]+', '', filename).strip(".mp3") +  "_" + str(i)  + ".mp3"
            self.single_split(i, i+min_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')
    

In [10]:


import os
import numpy as np
import librosa
track_durations = []
track_tempo = []
track_beat_frames= []
track_beat_times = []
#directory = r'C:\Users\abelf\Downloads\songs-upbeat - divided'
songs =  []
loudness_tracks = []

def calculate_features(directory,songs, track_tempo,loudness_tracks,track_durations):
    for filename in os.listdir(directory):

        try:
            y, sr = librosa.load(directory+"\\"+filename)
            songs.append(filename)
            # Compute the track duration
            track_duration = librosa.get_duration(y=y, sr=sr)
            track_durations.append(track_duration)
            # Ext'ract tempo and beat estimates
            onset_env = librosa.onset.onset_strength(y, sr=sr)
            tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
            track_tempo.append(tempo)
            # Compute the spectrogram (magnitude)
            spec_mag = abs(librosa.stft(y,n_fft = 2048,hop_length = 1024))
            # Convert the spectrogram into dB
            spec_db = librosa.amplitude_to_db(spec_mag)
            # Compute A-weighting values
            freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)
            a_weights = librosa.A_weighting(freqs)
            a_weights = np.expand_dims(a_weights, axis=1)

            # Apply the A-weghting to the spectrogram in dB
            spec_dba = spec_db + a_weights
            # Compute the "loudness" value
            loudness = librosa.feature.rms(y=y)
            
            for i in loudness:
                 summ = sum(i)/len(i)
                 loudness_tracks.append(summ)
            print(len(songs))
            print(len(track_tempo))
            print(len(track_durations))
            print(len(loudness_tracks))
        except:
            continue
def create_df(songs, track_tempo,track_durations, loudness_tracks):
     songs_df = {'songs':songs,
            'tempo':track_tempo,
            #'BPM': bpms,
            'Duration': track_durations,
            'Loudness':loudness_tracks
           
           }

     songs_df = pd.DataFrame(songs_df)
     
     return songs_df
    
            

In [78]:
'''
# generate random integer values
from random import seed
from random import randint
# seed random number generator
seed(1)
# generate some integers
#for _ in range(10):
	#value = randint(0, 10)
	#print(value)
lst = []
for col in songs_df["tempo"]:
       # print(col)
         if int(col) in range(65,75):
            lst.append(randint(65, 75)) 
         elif int(col) in range(76,85):
            lst.append(randint(75, 85))
         elif int(col) in range(86,94):
            lst.append(randint(86, 95))
         elif int(col) in range(95,110):
            lst.append(randint(96, 110))
         else:
            lst.append(randint(110, 120))
songs_df["heart_rate"] = lst
import openpyxl
songs_df.to_excel("heart_rate_1.xlsx")
'''

In [ ]:
import os
#directory = r'C:\Users\abelf\Downloads\songs_raw_data_ML-20211026T235932Z-001\songs_raw_data_ML'
import sys
user_input = r"C:\Users\abelf\Downloads\create_tempo"
#assert os.path.exists(user_input), "I did not find the file at, "+str(user_input)
track_durations = []
track_tempo = []
songs =  []
loudness_tracks = []
song_df = pd.DataFrame()

for filename in os.listdir(user_input):
    #if filename.endswith(".jpg") or filename.endswith(".png"):
    if '.mp3' in filename and '.wav' not in filename:
        print(filename)
        split_wav = SplitWavAudioMubin(user_input, filename)
        print("Splitting song {}...".format(filename))
        split_wav.multiple_split(1)
        print("Splitting done!\n")
        print("CALCULATE FEATURES")
        calculate_features(user_input,songs,track_tempo,loudness_tracks,track_durations)
        song_df = create_df(songs, track_tempo,track_durations,loudness_tracks)
        
print(song_df)
        

In [3]:

songs_df = pd.read_excel(r"C:\Users\abelf\Downloads\songs_raw_data_ML\songs_raw_data_ML\heart_rate.xlsx")
songs_df

,songs,tempo,Duration,Loudness,heart_rate
0,- Battleme - Burn This Town.mp57 - Battleme -...,56.173573,60.000000,0.032652,112
1,- Battleme - Burn This Town.mp57 - Battleme -...,56.173573,60.000000,0.033782,119
2,- Battleme - Burn This Town.mp57 - Battleme -...,56.173573,60.000000,0.033946,111
3,- Battleme - Burn This Town.mp57 - Battleme -...,117.453835,4.841995,0.000021,114
4,"- Battleme - Hey Hey, My My.mp11 - Battleme -...",80.749512,60.000000,0.035301,76
...,...,...,...,...,...
216,Vaults Vultures YG.mpVaults Vultures YG_3.mp3,112.347147,52.802993,0.070936,117
217,_Jump.mp11_Ju_0.mp3,103.359375,60.000000,0.081320,99
218,_Jump.mp11_Ju_1.mp3,103.359375,60.000000,0.080964,104
219,_Jump.mp11_Ju_2.mp3,103.359375,60.000000,0.082538,106


In [48]:
X = songs_df[["tempo","Loudness"]]
y = songs_df[["heart_rate"]]

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42, test_size = 0.2)

In [57]:
reg = LinearRegression().fit(X,y)

In [58]:
reg.score(X,y)

0.3943911599655293

In [ ]:
tempo_pre, loudness_pre = song_df["tempo"][0],song_df["Loudness"][0]

In [55]:
print("Predicted song heart rate : ", reg.predict([[tempo_pre, loudness_pre]]))

C:\Usersminiconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


array([[92.73417147]])

In [88]:
songs_df.to_csv("Songs_.csv")